In [10]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import xgboost as xgb

# Load datasets
train = pd.read_csv("/Users/micah/Desktop/preprocessed_train.csv")
test = pd.read_csv("/Users/micah/Desktop/preprocessed_test.csv")

# Inspect data
print(train.head())
print(test.head())


         ID  dri_score  psych_disturb  cyto_score  diabetes  hla_match_c_high  \
0 -1.731991   1.024391      -0.383754         NaN -0.427212          0.488048   
1 -1.731870  -0.562548      -0.383754         NaN -0.427212          0.488048   
2 -1.731750   1.024391      -0.383754         NaN -0.427212          0.488048   
3 -1.731630  -1.197324      -0.383754         NaN -0.427212          0.488048   
4 -1.731510  -1.197324      -0.383754         NaN -0.427212          0.488048   

   hla_high_res_8  tbi_status  arrhythmia  hla_low_res_6  ...  melphalan_dose  \
0        0.610215   -0.510911   -0.223156       0.649549  ...        0.580558   
1        0.610215    2.829815   -0.223156       0.649549  ...        0.580558   
2        0.610215   -0.510911   -0.223156       0.649549  ...        0.580558   
3        0.610215   -0.510911   -0.223156       0.649549  ...        0.580558   
4        0.610215   -0.510911   -0.223156       0.649549  ...       -1.722481   

   hla_low_res_8   cardiac

In [11]:
# Prepare features and targets
X = train.drop(columns=["ID", "efs", "efs_time"])  # Features
y = train["efs_time"]  # Use survival time as target for simplicity

# Split data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.2, random_state=42)


In [14]:
import xgboost as xgb

# Convert training data to DMatrix (XGBoost's native format)
dtrain = xgb.DMatrix(data=X_train, label=y_train)

# Define parameters
params = {
    "objective": "reg:squarederror",
    "random_state": 42,
}

# Train the model
xgb_model = xgb.train(
    params=params,
    dtrain=dtrain,
    num_boost_round=100
)

# Convert validation data to DMatrix for prediction
dval = xgb.DMatrix(data=X_val)
y_pred = xgb_model.predict(dval)


In [18]:
# Convert validation data to DMatrix format
dval = xgb.DMatrix(data=X_val)

# Predict on validation set
y_pred = xgb_model.predict(dval)

# Calculate Mean Squared Error
mse = mean_squared_error(y_val, y_pred)
print(f"Mean Squared Error: {mse}")


Mean Squared Error: 0.8717234711462457


In [19]:
# Convert the test data to DMatrix format
dtest = xgb.DMatrix(data=test.drop(columns=["ID"]))

# Generate predictions for the test dataset
test_predictions = xgb_model.predict(dtest)

# Create a DataFrame for submission
submission = pd.DataFrame({"ID": test["ID"], "prediction": test_predictions})

# Save the submission file
submission.to_csv("submission.csv", index=False)
print("Submission file saved as submission.csv")


Submission file saved as submission.csv


In [22]:
from sklearn.metrics import r2_score

# Calculate R^2 score as a temporary metric
r2 = r2_score(y_val, y_pred)
print(f"R^2 Score: {r2}")


R^2 Score: 0.1397905551345795
